In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, T5ForConditionalGeneration

c:\Users\Trevi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device_id = '0'
device = torch.device('cuda:'+device_id)

guideliner_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
guideliner = T5ForConditionalGeneration.from_pretrained("TrevorAshby/guideliner")

blen_tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
blen_model = AutoModelForSeq2SeqLM.from_pretrained("TrevorAshby/blenderbot-400M-distill")

guideliner.to(device)
blen_model.to(device)

c:\Users\Trevi\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Trevi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


BlenderbotForConditionalGeneration(
  (model): BlenderbotModel(
    (shared): Embedding(8008, 1280, padding_idx=0)
    (encoder): BlenderbotEncoder(
      (embed_tokens): Embedding(8008, 1280, padding_idx=0)
      (embed_positions): BlenderbotLearnedPositionalEmbedding(128, 1280)
      (layers): ModuleList(
        (0-1): 2 x BlenderbotEncoderLayer(
          (self_attn): BlenderbotAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
     

In [20]:
user_response="B: Hi! Yes, I am a huge fan of Star Wars! I also loved how it was the first major movie to be dubbed in Navajo! I think that is so cool! A: Yeah, that's wild. I guess it takes a pretty small audience to pay for the translation work."
topic_pref="{\"high-level\":{\"topic\":\"movie\", \"if-interest\":\"yes\"}, \"low-level\":{\"topic\":\"Star Wars\", \"if-interest\":\"yes\"}}"

In [21]:
guide_in_str = '{}| {}'.format(user_response, topic_pref)
in_ids = guideliner_tokenizer(guide_in_str, max_length=256, padding='max_length', return_tensors='pt').input_ids
guideline_example = guideliner.generate(in_ids.to(device), max_new_tokens=50)
guideline = guideliner_tokenizer.decode(guideline_example[0], skip_special_tokens=True)
print(guideline)

The user is a huge fan of Star Wars and have a high interest about it. Talk about their favorite scenes from the movie and suggest similar movies to watch.


In [22]:
blend_in_str = user_response
blend_in_str2 = ' [GUIDELINE] ' + guideline
blend_in_ids = blen_tokenizer([user_response + blend_in_str2], max_length=128, return_tensors='pt', truncation=True)
blend_example = blen_model.generate(**blend_in_ids.to(device), max_length=60)
blend_response = blen_tokenizer.batch_decode(blend_example, skip_special_tokens=True)[0]
generated_response = blend_response
print(generated_response)

 That's really cool! Do you have any other favorite movies or movies from the Star Wars franchise?
